# **Crypto Market Analysis**
This notebook analyzes historical price data for major cryptocurrencies (BTC, ETH, BNB, SOL) using data from Binance exchange.

In [14]:
#%% Import Libraries
import pandas as pd
import numpy as np

# import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# The "ccxt" library facilitates connection to various exchanges, including Binance
import ccxt

In [15]:
#%% Creating DataFrame

crypto = pd.DataFrame({
    'symbols': ["BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT", "AVAX/USDT"],
})

In [16]:
#%% Extracting Data from Binance

# Binance API configuration
binance = ccxt.binance()

# List of crypto to fetch data for
symbols = crypto['symbols'].tolist()  # Add other pairs as necessary
timeframe = '1M'  # Timeframe (1 month)

'''
Available timeframes:
'1m' - 1 minute
'1h' - 1 hour
'1d' - 1 day
'1w' - 1 week
'1M' - 1 month
'''

limit = 120  # Number of candles

# Create a list to store the data
all_data = []

# Loop to fetch data for each asset
for symbol in symbols:
    try:
        # Fetch OHLCV data from Binance
        OHLCV = binance.fetch_ohlcv(symbol, timeframe, limit=limit)
        
        print(f"\nSymbol: {symbol}")
        print(f"Number of candles retrieved: {len(OHLCV)}")
        print(f"Date range: {pd.to_datetime(OHLCV[0][0], unit='ms')} to {pd.to_datetime(OHLCV[-1][0], unit='ms')}")
        
        # Process all candles
        for candle in OHLCV:
            data = {
                'symbols': symbol,
                'timestamp': pd.to_datetime(candle[0], unit='ms'),
                'open': candle[1],
                'high': candle[2],
                'low': candle[3],
                'close': candle[4],
                'volume': candle[5]
            }
            all_data.append(data)
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

# Convert to a DataFrame
crypto_monthly_price = pd.DataFrame(all_data)

# Calculating Average Price
crypto_monthly_price['average_price'] = round((crypto_monthly_price['high'] + crypto_monthly_price['low']) / 2, 2)

# Group by month and calculate the average price
crypto_monthly_price['timestamp'] = pd.to_datetime(crypto_monthly_price['timestamp']).dt.to_period('M')

print(crypto_monthly_price.info())


Symbol: BTC/USDT
Number of candles retrieved: 94
Date range: 2017-08-01 00:00:00 to 2025-05-01 00:00:00

Symbol: ETH/USDT
Number of candles retrieved: 94
Date range: 2017-08-01 00:00:00 to 2025-05-01 00:00:00

Symbol: BNB/USDT
Number of candles retrieved: 91
Date range: 2017-11-01 00:00:00 to 2025-05-01 00:00:00

Symbol: SOL/USDT
Number of candles retrieved: 58
Date range: 2020-08-01 00:00:00 to 2025-05-01 00:00:00

Symbol: XRP/USDT
Number of candles retrieved: 85
Date range: 2018-05-01 00:00:00 to 2025-05-01 00:00:00

Symbol: AVAX/USDT
Number of candles retrieved: 57
Date range: 2020-09-01 00:00:00 to 2025-05-01 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype    
---  ------         --------------  -----    
 0   symbols        479 non-null    object   
 1   timestamp      479 non-null    period[M]
 2   open           479 non-null    float64  
 3   high           479 non-null   

In [17]:
#%% Grouping and Sorting Data

# Group by 'symbols' and 'timestamp' to get the average price for each month
crypto_monthly_avg = crypto_monthly_price[['symbols', 'timestamp', 'average_price']]

# Convert symbols to Categorical with specific order from the original crypto DataFrame
crypto_monthly_avg['symbols'] = pd.Categorical(
    crypto_monthly_avg['symbols'],
    categories=symbols,
    ordered=True
)

# Sort first by symbols (to group all records for each symbol together)
# Then by timestamp (descending) within each symbol group
crypto_monthly_avg = crypto_monthly_avg.sort_values(
    ['symbols', 'timestamp'], 
    ascending=[True, False]
).reset_index(drop=True)

print(crypto_monthly_avg.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype    
---  ------         --------------  -----    
 0   symbols        479 non-null    category 
 1   timestamp      479 non-null    period[M]
 2   average_price  479 non-null    float64  
dtypes: category(1), float64(1), period[M](1)
memory usage: 8.3 KB
None


C:\Users\Matheus\AppData\Local\Temp\ipykernel_1908\490511893.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
#%% Creating IBM Plex Sansactive Price Growth Graph

# Convert Period to datetime for plotting
crypto_monthly_avg['timestamp'] = crypto_monthly_avg['timestamp'].astype(str).apply(lambda x: pd.to_datetime(x))

# Create the figure
fig = px.line(
    crypto_monthly_avg,
    x='timestamp',
    y='average_price',
    color='symbols',
    custom_data=['symbols', 'average_price'],
    title='<b>Crypto Price Growth</b>',
    subtitle='Monthly Average | Log Scale',
    labels={
        'timestamp': 'Date',
        'average_price': 'Price (USD)',
        'symbols': 'Cryptocurrency'
    },
    color_discrete_map={
        'BTC/USDT': '#F7931A',  # Bitcoin Orange
        'ETH/USDT': '#636BFF',  # Ethereum Blue
        'BNB/USDT': '#F3BA2F',  # Binance Yellow
        'SOL/USDT': '#CF62E7',   # Solana Pink
        'XRP/USDT': '#4B8BBE',  # XRP Blue
        'AVAX/USDT': '#E84142'   # Avalanche Red
    },
    markers=False,
)

# Update traces to reduce marker size
fig.update_traces(
    marker={'size': 4}, # Try values between 2-6
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Price: $%{customdata[1]:,.2f}<br>" +
                  "<extra></extra>"
    )  
# Customize the layout
fig.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    height=435,  # Adjust height
    width=705,  # Adjust width
    xaxis_title="Date",
    yaxis_title="Price (USD)",
    yaxis_type="log"  # Log scale for better price comparison
)

# Add range slider
fig.update_xaxes(rangeslider_visible=True)

# Show the plot
fig.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

# Save the figure as an HTML file
fig.write_html("plots/html/cypto_price_growth.html")

In [48]:
#%% Creating IBM Plex Sansactive Price Growth Graph

# Convert Period to datetime for plotting
crypto_monthly_avg['timestamp'] = crypto_monthly_avg['timestamp'].astype(str).apply(lambda x: pd.to_datetime(x))

# Create the figure
figm = px.line(
    crypto_monthly_avg,
    x='timestamp',
    y='average_price',
    color='symbols',
    custom_data=['symbols', 'average_price'],
    title='Crypto Price Growth', 
    labels={
        'timestamp': 'Date',
        'average_price': 'Price (USD)',
        'symbols': 'Cryptocurrency'
    },
    color_discrete_map={
        'BTC/USDT': '#F7931A',  # Bitcoin Orange
        'ETH/USDT': '#636BFF',  # Ethereum Blue
        'BNB/USDT': '#F3BA2F',  # Binance Yellow
        'SOL/USDT': '#CF62E7',   # Solana Pink
        'XRP/USDT': '#4B8BBE',  # XRP Blue
        'AVAX/USDT': '#E84142'   # Avalanche Red
    },
    markers=False,
)

# Update traces to reduce marker size
figm.update_traces(
    marker={'size': 4}, # Try values between 2-6
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Price: $%{customdata[1]:,.2f}<br>" +
                  "<extra></extra>"
    )  
# Customize the layout
figm.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    title=dict(
        text='Crypto Price Growth',
        font=dict(size=14, color='white'),
        xanchor='left'
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    height=397,  # Adjust height
    width=353,  # Adjust width
    margin=dict(
        l=10,
        r=10,
        t=40,
        b=10 
    ),
    showlegend=False,
    xaxis_title="Date",
    yaxis_title="Price (USD)",
    yaxis_type="log"  # Log scale for better price comparison
)

# Show the plot
figm.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

# Save the figure as an HTML file
figm.write_html("plots/html/cypto_price_growth_mobile.html")

# **Crypto Relative Price Performance**
Each cryptocurrency price was normalized to 100% from July 2020 and then January 2024 to enable direct comparison of growth rates.

In [30]:
#%% Creating Normalized Price Data

# Create a copy of the DataFrame to avoid modifying the original
normalized_df_2020 = crypto_monthly_avg.copy()

# Convert timestamp to datetime if not already
normalized_df_2020['timestamp'] = pd.to_datetime(normalized_df_2020['timestamp'])

# Filter data starting from January 2024
start_date = pd.to_datetime('2020-07-01')
normalized_df_2020 = normalized_df_2020[normalized_df_2020['timestamp'] >= start_date]

# Sort by timestamp ascending to get correct base prices
normalized_df_2020 = normalized_df_2020.sort_values(['symbols', 'timestamp'], ascending=[True, True])

# Get the first value for each symbol
base_values = normalized_df_2020.groupby('symbols').first()['average_price']

# Calculate normalized prices 
normalized_df_2020['normalized_price'] = normalized_df_2020.apply(
    lambda x: (x['average_price'] / base_values[x['symbols']]) * 100,
    axis=1
)

C:\Users\Matheus\AppData\Local\Temp\ipykernel_1908\2269613118.py:17: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [20]:
#%% Create the normalized price evolution graph

# Create the figure
fig = px.line(
    normalized_df_2020,
    x='timestamp',
    y='normalized_price',
    color='symbols',
    custom_data=['symbols', 'average_price'],
    title='<b>Crypto Relative Price Performance</b>',
    subtitle='Normalized to July 2020',
    labels={
        'timestamp': 'Date',
        'normalized_price': 'Price Change (%)',
        'symbols': 'Cryptocurrency'
    },
    color_discrete_map={
        'BTC/USDT': '#F7931A',  # Bitcoin Orange
        'ETH/USDT': '#636BFF',  # Ethereum Blue
        'BNB/USDT': '#F3BA2F',  # Binance Yellow
        'SOL/USDT': '#CF62E7',   # Solana Pink
        'XRP/USDT': '#4B8BBE',  # XRP Blue
        'AVAX/USDT': '#E84142'  # Avalanche Red
    },
    markers=False,
)

# Update traces to reduce marker size
fig.update_traces(
    marker={'size': 4}, # Try values between 2-6
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Change: %{y:.1f}%<br>" +
                  "Price: $%{customdata[1]:,.2f}<br>" +
                  "<extra></extra>",
                  hoverlabel=dict(
    font=dict(
         family="IBM Plex Sans",
            size=11,
        ),
        bordercolor="rgba(17, 17, 17, 0.9)",  # Same as background to remove border
        namelength=-1  # Show full text
    ),
    )  

# Customize the layout
fig.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        title=dict(text="Cryptocurrency"),
        font=dict(
            family="IBM Plex Sans",
            size=11,
            color="white"
        ),
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05,
    ),
    height=435,  # Adjust height
    width=705,  # Adjust width
    xaxis_title="Date",
    yaxis_title="Price Change (%)",
    yaxis=dict(
        tickformat=',d', ticksuffix='%',  # Format y-axis as percentage    
    )
)

# Add range slider
fig.update_xaxes(rangeslider_visible=True)

# Show the plot
fig.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

# Save the figure as an HTML file
fig.write_html("plots/html/jul2020_crypto_relative_price_performance.html")

In [ ]:
#%% Create the normalized price evolution graph

# Create the figure
figm = px.line(
    normalized_df_2020,
    x='timestamp',
    y='normalized_price',
    color='symbols',
    custom_data=['symbols', 'average_price'],
    title='<b>Crypto Relative Price Performance</b>',
    subtitle='Normalized to July 2020',
    labels={
        'timestamp': 'Date',
        'normalized_price': 'Price Change (%)',
        'symbols': 'Cryptocurrency'
    },
    color_discrete_map={
        'BTC/USDT': '#F7931A',  # Bitcoin Orange
        'ETH/USDT': '#636BFF',  # Ethereum Blue
        'BNB/USDT': '#F3BA2F',  # Binance Yellow
        'SOL/USDT': '#CF62E7',   # Solana Pink
        'XRP/USDT': '#4B8BBE',  # XRP Blue
        'AVAX/USDT': '#E84142'  # Avalanche Red
    },
    markers=False,
)

# Update traces to reduce marker size
figm.update_traces(
    marker={'size': 4}, # Try values between 2-6
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Change: %{y:.1f}%<br>" +
                  "Price: $%{customdata[1]:,.2f}<br>" +
                  "<extra></extra>",
                  hoverlabel=dict(
    font=dict(
         family="IBM Plex Sans",
            size=11,
        ),
        bordercolor="rgba(17, 17, 17, 0.9)",  # Same as background to remove border
        namelength=-1  # Show full text
    ),
    )  

# Customize the layout
figm.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        title=dict(text="Cryptocurrency"),
        font=dict(
            family="IBM Plex Sans",
            size=11,
            color="white"
        ),
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05,
    ),
    showlegend=False,
    margin=dict(
        l=10,
        r=10,
        t=40,
        b=10 
    ),
    height=397,  # Adjust height
    width=353,  # Adjust width
    xaxis_title="Date",
    yaxis_title="Price Change (%)",
    yaxis=dict(
        tickformat=',d', ticksuffix='%',  # Format y-axis as percentage    
    )
)

# Add range slider
figm.update_xaxes(rangeslider_visible=True)

# Show the plot
figm.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

# Save the figure as an HTML file
fig.write_html("plots/html/jul2020_crypto_relative_price_performance_mobile.html")

In [21]:
#%% Creating Normalized Price Data

# Create a copy of the DataFrame to avoid modifying the original
normalized_df_2024 = crypto_monthly_avg.copy()

# Convert timestamp to datetime if not already
normalized_df_2024['timestamp'] = pd.to_datetime(normalized_df_2024['timestamp'])

# Filter data starting from January 2024
start_date = pd.to_datetime('2024-01-01')
normalized_df_2024 = normalized_df_2024[normalized_df_2024['timestamp'] >= start_date]

# Sort by timestamp ascending to get correct base prices
normalized_df_2024 = normalized_df_2024.sort_values(['symbols', 'timestamp'], ascending=[True, True])

# Get the first value for each symbol
base_values = normalized_df_2024.groupby('symbols').first()['average_price']

# Calculate normalized prices 
normalized_df_2024['normalized_price'] = normalized_df_2024.apply(
    lambda x: (x['average_price'] / base_values[x['symbols']]) * 100,
    axis=1
)

C:\Users\Matheus\AppData\Local\Temp\ipykernel_1908\1140205728.py:17: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [22]:
#%% Create the normalized price evolution graph

# Create the figure
fig = px.line(
    normalized_df_2024,
    x='timestamp',
    y='normalized_price',
    color='symbols',
    custom_data=['symbols', 'average_price'],
    title='<b>Crypto Relative Price Performance</b>',
    subtitle='Normalized to January 2024',
    labels={
        'timestamp': 'Date',
        'normalized_price': 'Price Change (%)',
        'symbols': 'Cryptocurrency'
    },
    color_discrete_map={
        'BTC/USDT': '#F7931A',  # Bitcoin Orange
        'ETH/USDT': '#636BFF',  # Ethereum Blue
        'BNB/USDT': '#F3BA2F',  # Binance Yellow
        'SOL/USDT': '#CF62E7',   # Solana Pink
        'XRP/USDT': '#4B8BBE',  # XRP Blue
        'AVAX/USDT': '#E84142'  # Avalanche Red
    },
    markers=True,
    
)

# Update traces to reduce marker size
fig.update_traces(
    marker={'size': 4}, # Try values between 2-6
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Change: %{y:.1f}%<br>" +
                  "Price: $%{customdata[1]:,.2f}<br>" +
                  "<extra></extra>"
    )  
# Customize the layout
fig.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    height=435,  # Adjust height
    width=705,  # Adjust width
    xaxis_title="Date",
    yaxis_title="Price Change (%)",
    yaxis=dict(tickformat=',d', ticksuffix='%')  # Format y-axis as percentage
)

# Add range slider
fig.update_xaxes(rangeslider_visible=True)

# Show the plot
fig.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

# Save the figure as an HTML file
fig.write_html("plots/html/jan2024_crypto_relative_price_performance.html")

In [23]:
#%% Creating Normalized Price Data

# Create a copy of the DataFrame to avoid modifying the original
normalized_df_2025 = crypto_monthly_avg.copy()

# Convert timestamp to datetime if not already
normalized_df_2025['timestamp'] = pd.to_datetime(normalized_df_2025['timestamp'])

# Filter data starting from January 2024
start_date = pd.to_datetime('2025-01-01')
normalized_df_2025 = normalized_df_2025[normalized_df_2025['timestamp'] >= start_date]

# Sort by timestamp ascending to get correct base prices
normalized_df_2025 = normalized_df_2025.sort_values(['symbols', 'timestamp'], ascending=[True, True])

# Get the first value for each symbol
base_values = normalized_df_2025.groupby('symbols').first()['average_price']

# Calculate normalized prices 
normalized_df_2025['normalized_price'] = normalized_df_2025.apply(
    lambda x: (x['average_price'] / base_values[x['symbols']]) * 100,
    axis=1
)

C:\Users\Matheus\AppData\Local\Temp\ipykernel_1908\893637716.py:17: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [24]:
#%% Create the normalized price evolution graph

# Create the figure
fig = px.line(
    normalized_df_2025,
    x='timestamp',
    y='normalized_price',
    color='symbols',
    title='<b>Crypto Relative Price Performance</b>',
    subtitle='Normalized to January 2025',
    labels={
        'timestamp': 'Date',
        'normalized_price': 'Price Change (%)',
        'symbols': 'Cryptocurrency'
    },
    color_discrete_map={
        'BTC/USDT': '#F7931A',  # Bitcoin Orange
        'ETH/USDT': '#636BFF',  # Ethereum Blue
        'BNB/USDT': '#F3BA2F',  # Binance Yellow
        'SOL/USDT': '#CF62E7',   # Solana Pink
        'XRP/USDT': '#4B8BBE',  # XRP Blue
        'AVAX/USDT': '#E84142'  # Avalanche Red
    },
    markers=True,
    
)

# Update traces to reduce marker size
fig.update_traces(marker={'size': 4})  # Try values between 2-6

# Customize the layout
fig.update_layout(
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    height=435,  # Adjust height
    width=705,  # Adjust width
    xaxis_title="Date",
    yaxis_title="Price Change (%)",
    yaxis=dict(tickformat=',d', ticksuffix='%')  # Format y-axis as percentage
)

# Add range slider
fig.update_xaxes(rangeslider_visible=True)



# Show the plot
fig.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})

# Save the figure as an HTML file
fig.write_html("plots/html/jan2025_crypto_relative_price_performance.html")

In [25]:
#%% Separating Symbols into Individual DataFrames
# Create individual DataFrames for each symbol and sort each DataFrame by timestamp in descending order
'''
btc_data = crypto_monthly_avg[crypto_monthly_price['symbols'] == 'BTC/USDT'].reset_index(drop=True)
btc_data = btc_data.sort_values('timestamp', ascending=False).reset_index(drop=True)

eth_data = crypto_monthly_avg[crypto_monthly_price['symbols'] == 'ETH/USDT'].reset_index(drop=True)
eth_data = eth_data.sort_values('timestamp', ascending=False).reset_index(drop=True)

bnb_data = crypto_monthly_avg[crypto_monthly_price['symbols'] == 'BNB/USDT'].reset_index(drop=True)
bnb_data = bnb_data.sort_values('timestamp', ascending=False).reset_index(drop=True)

sol_data = crypto_monthly_avg[crypto_monthly_price['symbols'] == 'SOL/USDT'].reset_index(drop=True)
sol_data = sol_data.sort_values('timestamp', ascending=False).reset_index(drop=True)

'''

"\nbtc_data = crypto_monthly_avg[crypto_monthly_price['symbols'] == 'BTC/USDT'].reset_index(drop=True)\nbtc_data = btc_data.sort_values('timestamp', ascending=False).reset_index(drop=True)\n\neth_data = crypto_monthly_avg[crypto_monthly_price['symbols'] == 'ETH/USDT'].reset_index(drop=True)\neth_data = eth_data.sort_values('timestamp', ascending=False).reset_index(drop=True)\n\nbnb_data = crypto_monthly_avg[crypto_monthly_price['symbols'] == 'BNB/USDT'].reset_index(drop=True)\nbnb_data = bnb_data.sort_values('timestamp', ascending=False).reset_index(drop=True)\n\nsol_data = crypto_monthly_avg[crypto_monthly_price['symbols'] == 'SOL/USDT'].reset_index(drop=True)\nsol_data = sol_data.sort_values('timestamp', ascending=False).reset_index(drop=True)\n\n"

In [26]:
#%% Creating IBM Plex Sansactive Area Chart with Price in Hover and Fullscreen

# Create the figure
fig = go.Figure()

# Color mapping for both fill and lines
color_map = {
    'BTC/USDT': {'fill': 'rgba(247, 147, 26, 0.2)', 'line': '#F7931A'},  # Bitcoin Orange
    'ETH/USDT': {'fill': 'rgba(99, 107, 255, 0.2)', 'line': '#636BFF'},  # Ethereum Blue
    'BNB/USDT': {'fill': 'rgba(243, 186, 47, 0.2)', 'line': '#F3BA2F'},  # Binance Yellow
    'SOL/USDT': {'fill': 'rgba(207, 98, 231, 0.2)', 'line': '#CF62E7'},  # Solana Pink
    'XRP/USDT': {'fill': 'rgba(75, 139, 190, 0.2)', 'line': '#4B8BBE'},  # XRP Blue
    'AVAX/USDT': {'fill': 'rgba(232, 65, 66, 0.2)', 'line': '#E84142'}   # Avalanche Red
}

# Add traces for each cryptocurrency in reverse order
for symbol in reversed(symbols):
    symbol_data = normalized_df_2024[normalized_df_2024['symbols'] == symbol]
    
    fig.add_trace(
        go.Scatter(
            x=symbol_data['timestamp'],
            y=symbol_data['normalized_price'],
            name=symbol,
            fill='tonexty',
            mode='lines',
            line=dict(width=1, color=color_map[symbol]['line']),
            fillcolor=color_map[symbol]['fill'],
            hovertemplate=(
                f"<b>{symbol}</b><br>" +
                "Change: %{y:.1f}%<br>" +
                "Price: $%{customdata:,.2f}<br>" +
                "<extra></extra>"
            ),
            customdata=symbol_data['average_price']
        )
    )

# Customize the layout
fig.update_layout(
    title=dict(
        text='<b>Cryptocurrency Performance - Normalized Price Change</b>',
        font=dict(size=24)
    ),
    template='plotly_dark',
    hovermode='x unified',
    font_family="IBM Plex Sans",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    xaxis_title="Date",
    yaxis_title="Price Change (%)",
    height=800/1.25,  # Adjust height
    width=1200/1.25,  # Adjust width
    showlegend=True,
    # Add configuration for modebar buttons
    modebar=dict(
        add=['fullscreen']
    ), 
)

# Update y-axis format
fig.update_yaxes(tickformat=',d', ticksuffix='%')

# Add range slider
fig.update_xaxes(rangeslider_visible=True)

# Show the plot with config options
fig.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})